In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2009)

In [2]:
midwest_data = pd.read_csv('MidwestMurderData.csv')
midwest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145080 entries, 0 to 145079
Data columns (total 32 columns):
ID              145080 non-null object
CNTYFIPS        145080 non-null object
Ori             145080 non-null object
State           145080 non-null object
Agency          145080 non-null object
Agentype        145080 non-null object
Source          145080 non-null object
Solved          145080 non-null object
Year            145080 non-null int64
StateName       145080 non-null object
Month           145080 non-null object
Incident        145080 non-null int64
ActionType      145080 non-null object
Homicide        145080 non-null object
Situation       145080 non-null object
VicAge          145080 non-null int64
VicSex          145080 non-null object
VicRace         145080 non-null object
VicEthnic       145080 non-null object
OffAge          145080 non-null int64
OffSex          145080 non-null object
OffRace         145080 non-null object
OffEthnic       145080 non-null obj

In [3]:
# midwest_17 = midwest_data.loc[(midwest_data['Year'] == 2017)]
# midwest_17.head(500)

In [4]:
midwest_data

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,197601001IA00801,"Boone, IA",IA00801,Iowa,Boone,Municipal police,FBI,No,1976,IOWA,...,Unknown or not reported,Knife or cutting instrument,Relationship not determined,Other arguments,NaN,0,0,30180.0,Iowa,Rural Iowa
1,197601001IA04400,"Henry, IA",IA04400,Iowa,Henry County,Sheriff,FBI,Yes,1976,IOWA,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Acquaintance,Robbery,NaN,0,0,30180.0,Iowa,Rural Iowa
2,197601001IA05202,"Johnson, IA",IA05202,Iowa,Iowa City,Municipal police,FBI,Yes,1976,IOWA,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Acquaintance,Other arguments,NaN,0,0,30180.0,Iowa,"Iowa City, IA"
3,197601001IA05701,"Linn, IA",IA05701,Iowa,Cedar Rapids,Municipal police,FBI,Yes,1976,IOWA,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Husband,Other arguments,NaN,0,0,30180.0,Iowa,"Cedar Rapids, IA"
4,197601001IA05703,"Linn, IA",IA05703,Iowa,Hiawatha,Municipal police,FBI,Yes,1976,IOWA,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Stranger,Brawl due to influence of alcohol,NaN,0,0,30180.0,Iowa,"Cedar Rapids, IA"
5,197601001IA06400,"Marshall, IA",IA06400,Iowa,Marshall County,Sheriff,FBI,Yes,1976,IOWA,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Acquaintance,Lovers triangle,NaN,0,0,30180.0,Iowa,Rural Iowa
6,197601001IA07800,"Pottawattamie, IA",IA07800,Iowa,Pottawattamie County,Sheriff,FBI,Yes,1976,IOWA,...,Unknown or not reported,Shotgun,Stranger,Other - not specified,NaN,0,4,30180.0,Iowa,"Omaha-Council Bluffs, NE-IA"
7,197601001IL00103,"Adams, IL",IL00103,Illinois,Quincy,Municipal police,FBI,Yes,1976,ILL,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Friend,Brawl due to influence of alcohol,NaN,0,1,30180.0,Illinois,Rural Illinois
8,197601001IL01000,"Champaign, IL",IL01000,Illinois,Champaign County,Sheriff,FBI,No,1976,ILL,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Narcotic drug laws,NaN,0,0,30180.0,Illinois,"Champaign-Urbana, IL"
9,197601001IL01600,"Cook, IL",IL01600,Illinois,Cook County,Sheriff,FBI,Yes,1976,ILL,...,Unknown or not reported,"Handgun - pistol, revolver, etc",Stranger,Robbery,NaN,0,1,30180.0,Illinois,"Chicago-Naperville-Joliet, IL-IN-WI"


In [63]:
acs_msa = c.acs5.get(('NAME', 'B01003_001E'), \
                    {'for': 'metropolitan statistical area/micropolitan statistical area:*'})
metrodf = pd.DataFrame(acs_msa)
metrodf = metrodf.rename(columns={"B01003_001E": "Population", "NAME": "MSA1",
                                  "metropolitan statistical area/micropolitan statistical area": "MSA Code"})
metro_only = metrodf[metrodf['MSA1'].str.contains('Metro Area')]
metro_only = metro_only.reset_index(drop=True)

# new data frame with split value columns to isolate 'Metro Area'
new_metro = metro_only["MSA1"].str.split(" Metro Area", n = 1, expand = True)  
# making separate MSA column from new data frame 
metro_only["MSA"]= new_metro[0] 
# Dropping old MSA columns 
metro_only.drop(columns =["MSA1"], inplace = True) 

# new data frame with split columns to isolate state abbreviations
metrostates = metro_only["MSA"].str.split(", ", n = 1, expand = True) 
# add separate state column from new df
metro_only['State'] = metrostates[1]

# create list of unique values in state column to identify all possible midwest state labels in MidwestMurderData.csv
stateslist = metro_only['State'].unique().tolist() 
midweststates = ['OH-PA', 'KS', 'WV-OH', 'WI', 'IA', 'OH', 'IN', 'MO', 'IL', 'IN-MI', 'SD', 'IA-NE-SD', \
                  'MO-IL', 'MO-KS', 'MN', 'MI', 'NE-IA', 'MN-WI', 'KY-IN', 'NE', 'WI-MN', 'WV-KY-OH', \
                  'ND-MN', 'AR-MO', 'IN-KY', 'IA-IL', 'OH-KY-IN', 'IL-IN-WI', 'ND']
# new dataframe with just state values in the midweststates list, reset index 
midwest_metros = metro_only[metro_only['State'].isin(midweststates)]
midwest_metros = midwest_metros.reset_index(drop=True)
midwest_metros

,Population,MSA Code,MSA,State
0,571519.0,49660,"Youngstown-Warren-Boardman, OH-PA",OH-PA
1,596643.0,48620,"Wichita, KS",KS
2,145942.0,48540,"Wheeling, WV-OH",WV-OH
3,122778.0,48260,"Weirton-Steubenville, WV-OH",WV-OH
4,129803.0,48140,"Wausau, WI",WI
5,163040.0,47940,"Waterloo-Cedar Falls, IA",IA
6,228761.0,45820,"Topeka, KS",KS
7,673545.0,45780,"Toledo, OH",OH
8,169733.0,45460,"Terre Haute, IN",IN
9,140419.0,44220,"Springfield, OH",OH


In [ ]:
# acs_msa_IN = c.acs5.get(('NAME', 'B01003_001E'), \
#                              {'for': 'metropolitan statistical area/micropolitan statistical area:*', 'in': 'state:18'})
# metrodf_IN = pd.DataFrame(acs_msa_IN)
# metrodf_IN = metrodf_IN.rename(columns={"B01003_001E": "Population",
#                                         "NAME": "MSA",
#                                         "metropolitan statistical area/micropolitan statistical area": "MSA Code"})

# acs_msa_IA = c.acs5.get(('NAME', 'B01003_001E'), \
#                              {'for': 'metropolitan statistical area/micropolitan statistical area:*', 'in': 'state:19'})
# metrodf_IA = pd.DataFrame(acs_msa_IA)
# metrodf_IA = metrodf_IA.rename(columns={"B01003_001E": "Population",
#                                         "NAME": "MSA",
#                                         "metropolitan statistical area/micropolitan statistical area": "MSA Code"})

,Population,MSA,MSA Code
206,2803776.0,"St. Louis, MO-IL Metro Area",41180


In [ ]:
# census_data_cntyfips = c.acs5.get(('NAME', 'B01003_001E', 'B19301_001E', 'B17001_002E'), {'for': 'county:*', 'in': 'state:*'})
# census_df_cntyfips = pd.DataFrame(census_data_cntyfips)
# census_df_cntyfips = census_df_cntyfips.rename(columns={"B01003_001E": "Population",
#                                                         "B19301_001E": "Per Capita Income",
#                                                         "B17001_002E": "Poverty Count",
#                                                         "NAME": "Name",
#                                                         "county": "County"})
# census_df_cntyfips
